# Divide video
The task in this project is to estimate strain in a basal segment by tracking two points between the end-diastole and end-systole frames in cardiac ultrasound videos. We assume that we have a method of detecting ED and ES frames in the ultrasound videos. Also, check that the points end up approximately in the same place. Then we need to divide the videos into clips from ED to ED

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import glob
import os
# Be able to import from src folder
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)
import sys
sys.path.append(os.path.join(PROJ_ROOT, 'src'))

import misc.notebookHelpers

%matplotlib inline
%config InlineBackend.figure_format = "retina"

plt.rcParams['figure.figsize'] = (10, 10)

In [ ]:
data_path = os.path.join(PROJ_ROOT, 'data/processed/ds_ma_labeled/')
file_paths = glob.glob(os.path.join(data_path, '*/*.h5'))

In [ ]:
file_paths[26]

In [ ]:
h5file = h5py.File(file_paths[26])
print([k for k in h5file['tissue'].keys()])

In [ ]:
names = []
h5file.visit(names.append)
names

In [ ]:
h5file['tissue/origin'][:]

Do a little quality check and see the first es and ed image

In [ ]:
first_ed = np.where(h5file['tissue/ds_labels'][:] == 1.0)[0][0]
first_ed

In [ ]:
first_es = np.where(h5file['tissue/ds_labels'][first_ed:] == 2.0)[0][0]
first_es

In [ ]:
video, fps = misc.notebookHelpers.video_and_fps_from_h5py(h5file)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(8, 16))
ax[0].imshow(video[first_ed, :, :], cmap='Greys_r')
ax[1].imshow(video[first_es + first_ed, :, :], cmap='Greys_r')

Looks all right, find the indicises of every ED with a following ES

In [ ]:
full_ed_idc = np.where(h5file['tissue/ds_labels'][:] == 1.0)[0]
ed_idc = []
es_idc = []

for ed_idx in full_ed_idc:
    remaining_es = np.where(h5file['tissue/ds_labels'][ed_idx:] == 2.0)[0]
    if remaining_es.size > 0:
        next_es = remaining_es[0] + ed_idx
        es_idc.append(next_es)
        ed_idc.append(ed_idx)
    else:
        break

In [ ]:
ed_idc

In [ ]:
es_idc

Check that we in fact have indices from the first ED to the last ES

In [ ]:
vis_arr = np.zeros(es_idc[-1] - ed_idc[0] + 1)
vis_arr[ed_idc - ed_idc[0]] = 1
vis_arr[es_idc - ed_idc[0]] = 2

plt.stem(vis_arr)

Try to view a clip for quality assurance

In [ ]:
clip = video[ed_idc[1]:ed_idc[2] + 1, :, :]
misc.notebookHelpers.ultraSoundAnimation(clip, fps=fps)

Looks good, now find corresponding TVI frames to keep them with the dataset

In [ ]:
tvi_ed_idc = []
for ed_idx in ed_idc:
    ed_time = h5file['tissue/times'][ed_idx]
    
    tvi_ed_idx = np.argmin(np.abs(h5file['TVI/times'] - ed_time))
    tvi_ed_idc.append(tvi_ed_idx)

In [ ]:
tvi_video, tvi_fps = misc.notebookHelpers.video_and_fps_from_h5py(h5file, 'tvi')
tvi_clip = tvi_video[tvi_ed_idc[1]:tvi_ed_idc[2] + 1, :, :]
misc.notebookHelpers.ultraSoundAnimation(tvi_clip, fps=tvi_fps)

In [ ]:
h5file['tissue/pixelsize'][:]

In [ ]:
print([k for k in h5file['tissue'].keys()])
print([k for k in h5file['TVI'].keys()])

In [ ]:
print([v for v in h5file['tissue/pixelsize']])

Try to make new files containing one video segment from ED to ES each. Add TVI, pixel information, directions, origin and timing for TVI and B-mode

In [ ]:
new_file_name_base = file_paths[10].split('/')[-1][:-3]
new_file_folder = '../data/interim/'
for i, _ in enumerate(ed_idc):
    new_file = h5py.File(
        os.path.join(new_file_folder, 
                     new_file_name_base + f'_{i}_.h5'), 
        'w')
    ed_idx = ed_idc[i]
    es_idx = es_idc[i]
    
    ed_time = h5file['tissue/times'][ed_idx]
    es_time = h5file['tissue/times'][es_idx]
    
    video = h5file['tissue/data'][:]
    clip = video[:, :, ed_idx:es_idx + 1]
    clip_times = h5file['tissue/times'][ed_idx:es_idx + 1]
    
    tissue = new_file.create_group('tissue')
    tissue.create_dataset('data', data=clip)
    tissue.create_dataset('times', data=clip_times)
    tissue.create_dataset('dirx', data=h5file['tissue/dirx'])
    tissue.create_dataset('diry', data=h5file['tissue/diry'])
    tissue.create_dataset('pixelsize', data=h5file['tissue/pixelsize'])
    tissue.create_dataset('origin', data=h5file['tissue/origin'])
    tissue.create_dataset('mitral_points', data=h5file['tissue/mitral_points'])
    try:
        tvi_ed_idx = np.argmin(np.abs(h5file['TVI/times'] - ed_time))
        tvi_es_idx = np.argmin(np.abs(h5file['TVI/times'] - es_time))
        
        tvi_video = h5file['TVI/data'][:]
        tvi_clip = tvi_video[:, :, tvi_ed_idx:tvi_es_idx + 1]
        tvi_clip_times = h5file['TVI/times'][ed_idx:es_idx + 1]
        
        tvi = new_file.create_group('TVI')
        tvi.create_dataset('data', data=tvi_clip)
        tvi.create_dataset('times', data=tvi_clip_times)
        # Directions swapped after transpose
        tvi.create_dataset('dirx', data=h5file['TVI/diry'])
        tvi.create_dataset('diry', data=h5file['TVI/dirx'])
        tvi.create_dataset('pixelsize', data=h5file['TVI/pixelsize'])
        tvi.create_dataset('origin', data=h5file['TVI/origin'])
    except:
        print(f'{new_file_name_base} is missing TVI/times, skip TVI for this file')
    new_file.close()

Try to open one of them to check that it went according to plan

In [ ]:
test_file = h5py.File(os.path.join(new_file_folder, 
                                      new_file_name_base + f'_{3}_.h5'))

In [ ]:
misc.ultraSoundAnimation(test_file['tissue/data'], fps=24)

In [ ]:
misc.ultraSoundAnimation(test_file['TVI/data'], fps=24)